Creating the kinetic data curve fitting script

Importing the data

In [6]:
import pandas as pd
import numpy as np
import os


#Import Data + Clean Data
def importCleanData(file_name, sheet, columns):
    data = pd.read_excel(file_name, sheet_name = sheet)
    cleanData = data.loc[:, columns].dropna()
    return cleanData

testFile105 = importCleanData("VEGFA165_NRP1_0.5-8nM_Lu2023.xlsx", "sheet1", ["Time 0.5nM", "RU 0.5nM"])

#Function Defining

def fit_functions_ssr_base(data):
    yi = 0
    yf = 0
    kon = 0
    return yf * (1 - np.exp(-kon * t)) + yi

def fit_function_zero(data):
    yi = 0
    yf = 0
    kon = 0
    koff = 0
    return (C / (kon - koff)) * (np.exp(-koff * t) - np.exp(-kon * t)) +yi

def fit_function_ssr(data):
    yi = 0
    yf = 0
    kon = 0
    koff = 0
    return yf * ((1 - D * math.exp(-kon * t)) + (D - 1) * math.exp(-koff * t)) + yi

#Data Reporting

def plot_data():
    plt.scatter()
    plt.plot()

    plt.figure()
    plt.title()
    plt.show()

def data_stats():
    np.mean()
    np.ptp()
    np.std()



IndentationError: expected an indented block after function definition on line 36 (2117057563.py, line 44)

Defining the functions

Reporting the data